# DATABASE CREATION
*Cristina Mendoza*

### Creation of a .csv file combining the downloaded tables from ADAS 3D

In [ ]:
# import libraries
import os
import pandas as pd
import numpy as np


# folder with all the cases (nhc)
input_folder = # only the cases (nhc) must be in the input folder
output_folder =  # It must be a different folder from the input. There should be the csv with the IDs and it is were the csv file will be saved.

In [ ]:
def csv_to_df(input_dir, output_dir):
    # function to access the csv files of each patient and join them into a single dataframe

    # list to save the final dataframe (will all patients)
    all_patients_df = []
    
    for patient in os.listdir(input_dir):  # enter the folder of the nhc of the patient
        patient_dir = os.path.join(input_dir, patient).replace("\\","/") # windows uses "/" (it has to be changed)
        
        patient_dfs_raw = [] # list to save the dfs (raw)
        patient_files = [] # list to save the names of the csv files

        # Check if "PRE" directory exists
        if not any(case.upper().startswith('PRE') for case in os.listdir(patient_dir)):
            print(f"No 'PRE' directory found for patient: {patient}")
            continue

        for case in os.listdir(patient_dir): # enter the folder of the study (if it starts with PRE or pre)
            # if it starts with PRE or pre
            if case.upper().startswith('PRE'):
                case_dir = os.path.join(patient_dir, case).replace("\\","/")

                # Check if "CSV" directory exists
                if not any(folder.upper().startswith("CSV") for folder in os.listdir(case_dir)):
                    print(f"No 'CSV' directory found for patient: {patient}")
                    continue

                # Enter the folder starting with CSV
                for folder in os.listdir(case_dir):
                    if folder.upper().startswith("CSV"):
                        csv_dir = os.path.join(case_dir, folder).replace("\\", "/")
                        
                        # list with the names of the csv files of each patient
                        csv_files = [file for file in os.listdir(csv_dir) if file.endswith('.csv')]
                        patient_files.extend(csv_files)
                        
                        # list with the dfs of each patient
                        csv_df_list = [pd.read_csv(os.path.join(csv_dir, file), delimiter=';') for file in csv_files]
                        patient_dfs_raw.extend(csv_df_list)
                        
        # dictionary with the name of the file and the dataframe of each patient
        csv_dict = dict(zip(patient_files, patient_dfs_raw))
        
        # Exclude the data from "numerical" dataframes (not relevant for structural and morphological atrial remodeling)
        csv_dict = {k: v for k, v in csv_dict.items() if "Numerical" not in k}

        # Exclude patients without csv
        if not csv_dict:
            continue
        
        # Take the NHC from each patient from the 1st key of its dictionary
        patient_number = list(csv_dict.keys())[0].split("_")[0] 

        #Create an empty dictionary to include the dfs of each patient
        final_dict = {}
        
        # create new variable names
        for key, df in csv_dict.items():
            chamber_name = key.split("_")[1] # Take the name of the chamber (LA or RA) from the keys
            df.set_index(df.columns[0], inplace = True) # Convert the first column into indices (row names)
            flattened_df = df.stack() # Flatten the dataframes into a single row

            for (row_name, col_name), value in flattened_df.items():
                new_var_name = f"{chamber_name}_{row_name}_{col_name}" # Create the new variable name
                final_dict[new_var_name] = value # Add the new variable to the dictionary
        
        # Convert the dictionary into a dataframe
        patient_df_flat = pd.DataFrame(final_dict, index = [patient_number])
        # Substitute the unwanted characters from the variables and convert numerical data into float type data
        patient_df_flat.rename(columns = lambda x: x.replace(" ", "_"), inplace = True) # change " " by "_"
        patient_df_flat.rename(columns = lambda x: x.replace("__","_"), inplace = True) # change "__" by "_"
        for col in patient_df_flat.columns:
            if patient_df_flat[col].dtype == "object":
                patient_df_flat[col] = patient_df_flat[col].str.replace(",",".").astype(float) # change "," by "." and convert to float
        
        all_patients_df.append(patient_df_flat)

    # Concatenate the rows of all patients vertically
    final_df = pd.concat(all_patients_df, axis = 0, ignore_index = False)
    final_df['ID']= np.nan

    # Make the indices of int type to allow correspondency to the ID dataframe
    final_df.index = final_df.index.astype(int)
    
    # Add the column of the patient's ID (to sort them like the excel clinical database)
    id_df = pd.read_csv(os.path.join(output_dir, "id_csv.csv").replace("\\","/"), delimiter = ";", header = None)
    
    id_df.columns = ['NHC', 'ID']

    for ind, nhc in enumerate(id_df['NHC']):
        if nhc in final_df.index:
            final_df['ID'][nhc] = id_df['ID'][ind]

    # sort by ID (as in the excel)
    final_df.sort_values(by = 'ID', inplace = True)

    # Make the ID column become the first column
    cols = ['ID'] + [col for col in final_df.columns if col != 'ID'] 
    final_df = final_df[cols]
    final_df['ID'] = final_df['ID'].astype(int) # Convert ID variable to integer
 
      
    return final_df              

In [ ]:
# execute the function for the given directories
final_df = csv_to_df(input_folder, output_folder)
final_df

### Create new columns

- From the structural remodeling data, create new variables corresponding to the body of the atria (by subtracting the values of the structures like valves and veins from the total area). Recalculate the percentages too.

- Create new variables for the regions. (this is for future research related to this study)
    - Posterior wall: 3a+3b+3c+3d
    - Floor: 7a+7b+7c+7d
    - Lateral wall: 6
    - Anterior wall: 4a+4b+4c+4d
    - Septal wall: 8
    - Carines: 1a+1b+2

**New variables for fibrosis**

In [ ]:
inicis_list = ['RA_0%', 'LA_0%']
finals_list = ["_Total_Area_(cm2)","_BZ+Core_(cm2)", "_BZ_(cm2)", "_Core_(cm2)", "_Invalid_(cm2)"]

# Create the new variables for "body" by subtracting the different structures from the total
new_cols_list = []

for inici in inicis_list:
    for final in finals_list:
        variables_resta = [name for name in final_df.columns if name.startswith(inici) and name.endswith(final)]
        variables_resta = [element for element in variables_resta if "layer" not in element]
        variables_resta = [element for element in variables_resta if "body" not in element]
        nova_col = inici + '_body' + final
        new_cols_list.append(nova_col)
        final_df[nova_col] = final_df[inici + "_layer" + final] - final_df[variables_resta].sum(axis = 1)
        
        
# Create the new variables for "body" in %:

# list without Total_Area
new_cols_list_no_total = [element for element in new_cols_list if "Total" not in element]
# list of new variables (%):
new_cols_perc_list = [element.replace("cm2","%") for element in new_cols_list_no_total]
new_cols_perc_list

for ind, col in enumerate(new_cols_perc_list):
    cambra = col.split("_")[0]
    final_df[col] = round(final_df[new_cols_list_no_total[ind]]/final_df[cambra + "_0%_body_Total_Area_(cm2)"]*100,2)

 
final_df

**New variables for the LA regions:**

- Create new variables by grouping the regions (numerical):
    - Posterior wall: 3a+3b+3c+3d
    - Floor: 7a+7b+7c+7d
    - Lateral wall: 6
    - Anterior wall: 4a+4b+4c+4d
    - Septal wall: 8
    - Carines: 1a+1b+2

In [ ]:
# List of new variables

new_regions = ["Carina", "PosteriorWall", "AneriorWall", "LateralWall", "Floor", "SeptalWall"]
new_regions_num = [["1","2"],["3"],["4"],["6"],["7"],["8"]]
finals_list = ["_Total_Area_(cm2)","_BZ+Core_(cm2)", "_BZ_(cm2)", "_Core_(cm2)", "_Invalid_(cm2)"]

new_cols_list = []

for ind, region in enumerate(new_regions):
    for final in finals_list:
        new_col = "LA_" + region + final
        new_cols_list.append(new_col)

new_cols_list

In [ ]:
# Dictionary of variables that we want to sum (numerical regions)

new_regions = ["Carina", "PosteriorWall", "AneriorWall", "LateralWall", "Floor", "SeptalWall"]
new_regions_num = [["1","2"],["3"],["4"],["6"],["7"],["8"]]
finals_list = ["_Total_Area_(cm2)","_BZ+Core_(cm2)", "_BZ_(cm2)", "_Core_(cm2)", "_Invalid_(cm2)"]

dict_old_cols = {}
for ind, region in enumerate(new_regions):
    region_list = []
    for col in final_df.columns:
        for val in col.split("_")[1:]:
            for i in new_regions_num[ind]:
                if val.startswith(i) == True:
                    region_list.append(col)
    dict_old_cols[region] = region_list

dict_old_cols

In [ ]:
# Dictionary with the new variables (keys) and the variables that conform them (values)
new_cols_dict = {}
for new_col in new_cols_list:
    for key in dict_old_cols.keys():
        if new_col.split("_")[1] == key:
            values = dict_old_cols.get(key)
            var_list = []
            for var in values:
                if new_col.split("_")[2:] == var.split("_")[2:]:
                    var_list.append(var)
                    new_cols_dict[new_col] = var_list
                    
new_cols_dict

In [ ]:
final_df = final_df
final_df = final_df.assign(**{key:final_df[columns].sum(axis = 1) for key, columns in new_cols_dict.items()})
final_df

In [ ]:
# Create the new variables for the regions in %:

# list without "Total_Area"
new_cols_list_no_total = [element for element in new_cols_list if "Total" not in element]
# list of new variables (%):
new_cols_perc_list = [element.replace("cm2","%") for element in new_cols_list_no_total]
new_cols_perc_list

for ind, col in enumerate(new_cols_perc_list):
    regio = col.split("_")[1]
    denom = final_df['LA_' + regio + "_Total_Area_(cm2)"]
    final_df[col] = np.where(denom != 0, round(final_df[new_cols_list_no_total[ind]] / denom * 100, 2), np.nan)
final_df

### Save the ADAS 3D variables into a csv file

In [ ]:
adas_df = final_df
adas_df.to_csv(os.path.join(output_folder, "final_df_adas.csv").replace("\\","/"), float_format='%.2f', index = False)

### Open the Excel's database (clinical data + tricuspid measurements)

In [ ]:
excel_dir =  #directory where the excel database is stored
excel_df = pd.read_csv(excel_dir, delimiter=';')

# set the NHC as the indices (like in the adas database)
excel_df.set_index('NHC', inplace = True)
excel_df.index.name = None
excel_df.columns = excel_df.columns.str.replace(" ","_") # substitute the " " in the database by "_"
excel_df

Create the new featue corresponding to the eccentricity index of the TA:

In [ ]:
excel_df["Eccentricity_1"] = excel_df["DMIN_PRE_ADASmm"]/excel_df["DMAX_PRE_ADAS_mm"]

### Concatenate the dataframes to unify the databases

In [ ]:
# Delete the ID column from the ADAS 3D dataframe because it is already in the Excel database.
adas_df = adas_df.drop('ID', axis = 1)

# Concatenate the two dataframes to create the final database.
bbdd_final = pd.concat([excel_df,adas_df], axis = 1)

bbdd_final

Substitute % by "perc" because the % simbol disappears in the csv.


It will also be change in the variables containing 0% layer

In [ ]:
bbdd_final.columns = bbdd_final.columns.str.replace("%","perc")

### Save the final database as a csv file

In [ ]:
bbdd_final.to_csv(os.path.join(output_folder, "final_df_all.csv").replace("\\","/"), float_format='%.2f')